In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import casadi as ca

In [53]:
def h1(x):
    return ca.vertcat(*[ca.norm_2(x[1]**2+x[2]**2-1),
                        ca.norm_2(x[0]-0)])
    

#def h2(x):
#    return ca.norm_2(x-np.array([0,1,0]))



In [67]:
#h1(np.array([1.1, 3.1, 1.1]))
x_sym = ca.SX.sym('x_sym',3)
h1_sym = h1(x_sym)
h2_sym = h2(x_sym)
h1_jac = ca.jacobian(h1_sym, x_sym)
h2_jac = ca.jacobian(h2_sym, x_sym)
h1_jac_fn = ca.Function('h1_jac', [x_sym],[h1_jac])
h2_jac_fn = ca.Function('h2_jac', [x_sym],[h2_jac])
print(h1_sym)
print(h2_sym)
print(h1_jac_fn(np.array([1, 1, 0.03])))

h1_jac_pinv = ca.pinv(h1_jac)
h1_jac_pinv_fn = ca.Function('h1_jac_pinv_fn', [x_sym], [h1_jac_pinv])

h2_jac_pinv = ca.pinv(h2_jac)
h2_jac_pinv_fn = ca.Function('h2_jac_pinv_fn', [x_sym], [h2_jac_pinv])


[fabs(((sq(x_sym_1)+sq(x_sym_2))-1)), fabs(x_sym_0)]
sqrt(((sq(x_sym_0)+sq((x_sym_1-1)))+sq(x_sym_2)))

[[00, 2, 0.06], 
 [1, 00, 00]]


In [85]:
#A*x=b
#J*lam=F

F = np.array([2, 1, 1])
x = np.array([2,0,1.3])
pi1_val = h1_jac_pinv_fn(x).full()
pi2_val = h2_jac_pinv_fn(x).full()

#print(h1_jac_fn(x))
#print(pi_val.size)
#print(F.size)
#print(h1_jac_fn(x).full().shape)

print(ca.norm_2(F-h1_jac_fn(x).T@(F@pi1_val)).full())
#print(ca.norm_2(F-h2_jac_fn(x)@(F@pi2_val)).full())


[[1.]]
